# Report

## Introduction and data

> REMOVE THE FOLLOWING TEXT

This section includes an introduction to the project motivation, data, and research question.
Describe the data and definitions of key variables.

It should also include some exploratory data analysis.

All of the EDA won't fit in the paper, so focus on the EDA for the response variable and a few other interesting variables and relationships.

### Project Motivation
1999 wurde in der Forschung (Collins et al.) ermittelt, dass die ASA-Klassifizierung des körperlichen Zustands, das Alter, Bluttransfusionen während der Operation und die Dauer von Operationen einen Aufschluss darüber geben, ob der Krankenhausaufenthalt des Patienten länger als gewöhnlich ausfallen wird.
Des Weiteren gibt es zahlreiche Studien, bspw. die Forschung von Biber et al. (2012), Singler et al. (2013) oder Ogliari et al. (2022) die festgestellt haben, dass es eine Korrelation zwischen dem Alter von Patienten und der Verweildauer in Notaufnahmen gab.
Dieser Frage möchten wir gerne nachgehen und versuchen, die Aufenthaltsdauer vorherzusagen

### Data
Uns liegen Daten von nicht-kardiochirurgischen Patienten vor, welche sich von August 2016 bis Juni 2017 einer Routine- oder Notoperation am Seoul National University Hospital, Seoul, Korea, unterzogen. Von den 7.051 in Frage kommenden Fällen wurden Fälle mit lokaler Anästhesie (239), unvollständiger Aufzeichnung (279) und Verlust von wesentlichen Datenspuren (145) ausgeschlossen. Schließlich wurden 6.388 Fälle (91 %), die eine Allgemeinanästhesie, Spinalanästhesie und Sedierung/Analgesie erhielten, in den Datensatz aufgenommen.
Der Datensatz besteht aus intraoperativen Vitaldaten und perioperativen klinischen Informationen von 6.388 Fällen.
Die Vitaldaten umfassen bis zu 12 Wellenform- und 184 numerische Datenspuren, die von mehreren Anästhesiegeräten erfasst wurden, die den Patienten während der Operation eingesetzt wurden.
Die Daten wurden nicht vorverarbeitet, da das reale Rauschen in den Vitaldaten für die Entwicklung praktischer Überwachungsalgorithmen sehr wichtig ist.
Insgesamt 74 perioperative klinische Informationsparameter und 34 Zeitreihen perioperativer Laborergebnisse werden zur Verfügung gestellt, um die Interpretation der Beziehung zu den intraoperativen Vitalzeichen zu erleichtern.

### Research Question
Wir stellen uns die Frage, welche dem Krankenhaus vorliegenden Daten die Aufenthaltsdauer von Patienten beeinflusst. Da der Datensatz des Seoul National University Hospital (Seoul, Korea) eine,Vielzahl von möglichen Variablen enthält werden wir uns vorerst auf den Zusammenhang zwischen dem Alter der Patienten und der Behandlungsdauer beschäftigen.

In [22]:
import pandas as pd
import altair as alt
import numpy as np
import joblib
import seaborn as sns


from sklearn.feature_selection import SelectFromModel

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.inspection import permutation_importance

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

### Data preparation

Wir berechnen eine weitere Variable "op_duartion". Sie zeigt die Dauer einer Operation an. Dafür subtrahieren wir von der Variablen "opend" die Variable "opstart".

In [24]:
df['op_duration']=df['opend']-df['opstart']

Wir benennen einige Variablen um:
"caseend" bezeichnet das Ende eines Falls, vom Startpunkt 0, daher bennen wir diese Variable "record_duration"
Die Variable "icu_days" bezeichnet die Aufenthaltsdauer im Krankenhaus (icu), daher bennen wir diese Variable "length_of_stay"
Da wir uns unter der Variable "intraop_ebl" wenig vorstellen konnten, benennen wir diese in "estimated_blood_loss" um.

In [25]:
df=df.rename(columns={'caseend':'recording_duration','icu_days':'length_of_stay','intraop_ebl':'estimated_blood_loss'})

Wir ändern nun den Datentyp der Variable "estimated_blood_loss", sodass diese korrekt erkannt wird.

In [26]:
df['estimated_blood_loss'] = df['estimated_blood_loss'].fillna(0).astype(int)

Die Spalte 'age' enthält u.a. die Zeichen ">89" und "0.7". 
Daher wird das Größer-als-Zeichen entfernt und die Werte gerundet, damit es zu int umwandelt werden kann. 

In [27]:
df['age'] = df['age'].str.replace('>89', '89').astype(float).round().astype(int)

Wir prüfen unsere Daten auf Zellen ohne Werte

In [52]:
print("Anzahl missng values in Spalte 'estimated_blood_loss':")
df['estimated_blood_loss'].isnull().sum()

Anzahl missng values in Spalte 'estimated_blood_loss':


0

In [51]:
print("Anzahl missng values in Spalte 'op_duration':")
df['op_duration'].isnull().sum()

Anzahl missng values in Spalte 'op_duration':


0

In [44]:
print("Anzahl missng values in Spalte 'length_of_stay':")
df['length_of_stay'].isnull().sum()

Anzahl missng values in Spalte 'length_of_stay':


0

In [45]:
print("Anzahl missng values in Spalte 'bmi':")
df['bmi'].isnull().sum()

Anzahl missng values in Spalte 'bmi':


0

In [46]:
print("Anzahl missng values in Spalte 'asa':")
df['asa'].isnull().sum()

Anzahl missng values in Spalte 'asa':


0

In [47]:
print("Anzahl missng values in Spalte 'age':")
df['age'].isnull().sum()

Anzahl missng values in Spalte 'age':


0

In [48]:
print("Anzahl missng values in Spalte 'recording_duration':")
df['recording_duration'].isnull().sum()

Anzahl missng values in Spalte 'recording_duration':


0

Lediglich die Spalte 'asa' hat 133 NAs. Diese werden durch den Median der Spalte ersetzt. 

In [35]:
df['asa'] = df['asa'].fillna(df['asa'].median())

Wir prüfen, ob der Vorgang korrekt durchgeführt wurde.

In [36]:
df['asa'].isnull().sum()

0

Aufgrund unserer Projektmotivation heraus nutzen wir die dort erwähnten Variablen. Zusätzlich nehmen wir noch die Variable "bmi" hinzu. Diese wird zwar nicht erwähnt, zeigt jedoch auf eine andere Art als die ASA-Klassifizierung (Variable "asa") auch den Körperlichen Zustand eines Patienten.
Aus der Projektmotivation ist auch zu entnehmen, dass wir die Krankenhausaufenhaltsdauer anhand dieser Prädiktoren bestimmen möchten.

In [37]:
df_Variables1=df[['recording_duration','age','asa','bmi','length_of_stay','op_duration','estimated_blood_loss']]

y_label = 'length_of_stay'

features = ['recording_duration',
            'age',
            'asa',
            'bmi',
            'op_duration',
            'estimated_blood_loss']

X = df_Variables1[features]
y = df_Variables1[y_label]

In [38]:
# inspect correlation between outcome and possible predictors
corr = df_train.corr()
corr[y_label].sort_values(ascending=False)

NameError: name 'df_train' is not defined

Im folgenden werden wir die Interkorrelation unter den Prädiktoren betrachten:

In [ ]:
# Inspect correlation
# Calculate correlation using the default method ( "pearson")
corr = df_train.corr()
# optimize aesthetics: generate mask for removing duplicate / unnecessary info
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
# Generate a custom diverging colormap as indicator for correlations:
cmap = sns.diverging_palette(220, 10, as_cmap=True)
# Plot
sns.heatmap(corr, mask=mask, cmap=cmap, annot=True,  square=True, annot_kws={"size": 12});

## Methodology

> REMOVE THE FOLLOWING TEXT

This section includes a brief description of your modeling process.

Explain the reasoning for the type of model you're fitting, predictor variables considered for the model.

Additionally, show how you arrived at the final model by describing the model selection process, variable transformations (if needed), assessment of conditions and diagnostics, and any other relevant considerations that were part of the model fitting process.

## Results

> REMOVE THE FOLLOWING TEXT

This is where you will output the final model with any relevant model fit statistics.

Describe the key results from the model.
The goal is not to interpret every single variable in the model but rather to show that you are proficient in using the model output to address the research questions, using the interpretations to support your conclusions.

Focus on the variables that help you answer the research question and that provide relevant context for the reader.


## Discussion + Conclusion


> REMOVE THE FOLLOWING TEXT

In this section you'll include a summary of what you have learned about your research question along with statistical arguments supporting your conclusions.
In addition, discuss the limitations of your analysis and provide suggestions on ways the analysis could be improved.
Any potential issues pertaining to the reliability and validity of your data and appropriateness of the statistical analysis should also be discussed here.
Lastly, this section will include ideas for future work.